HOSPITAL PATIENT DETERIORATION PREDICTION
Scenario: ICU Patient Health Risk Forecasting
A hospital ICU continuously monitors critical patient vitals.
The goal is to predict the patient’s risk score for the next 6 hours using the previous 12 hours of multivariate time-series data, so doctors can intervene before a medical emergency occurs.

RNN

In [65]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [66]:
lookback = 12
horizon = 6

df = pd.read_csv('vitals_dataset_48h_with_risk.csv')
features = ["HR", "BP", "O2", "Resp", "Temp"]
target = 'risk_score'
print(df.head())
print(len(df))

   Hour  HR   BP  O2  Resp  Temp  risk_score
0     1  82  120  98    16  36.8    0.000000
1     2  85  118  97    17  36.9    6.504384
2     3  88  115  96    18  37.0   13.580608
3     4  92  112  95    19  37.1   21.454831
4     5  96  108  94    20  37.2   29.900894
48


In [67]:
def make_window(df,features,target,lookback=12,horizon=6):
    Xv= df[features].values
    yv= df[target].values

    x_list,y_list = [],[]

    for i in range(len(df) - lookback - horizon +1):
        x_list.append(Xv[i:i+lookback])
        y_list.append(yv[i+lookback:i+lookback+horizon])
    return np.array(x_list),np.array(y_list)

In [68]:
X,y = make_window(df,features,target,lookback,horizon)
n = len(X)
split = int(0.8*n)

X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [69]:
x_scaler = StandardScaler()
N, T, F = X_train.shape

X_train_2d = X_train.reshape(-1, F)
X_val_2d   = X_val.reshape(-1, F)

x_scaler.fit(X_train_2d)
X_train = x_scaler.transform(X_train_2d).reshape(N, T, F)
X_val   = x_scaler.transform(X_val_2d).reshape(X_val.shape[0], T, F)


y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
y_val_scaled   = y_scaler.transform(y_val.reshape(-1, 1)).reshape(y_val.shape)


In [70]:
def build_model(input_shape, horizon):
    model = Sequential([
        SimpleRNN(64, activation="tanh", input_shape=input_shape),
        Dropout(0.25),
        Dense(32, activation="relu"),
        Dense(horizon)
    ])
    return model


In [71]:

def train_and_report(optimizer, label=""):
    tf.random.set_seed(1)

    model = build_model((lookback, F), horizon)

    # Huber loss reduces sensitivity to outliers/spikes
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.Huber(delta=1.0),
        metrics=["mae"]
    )

    es = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

    history = model.fit(
        X_train, y_train_scaled,
        validation_data=(X_val, y_val_scaled),
        epochs=200,
        batch_size=8,
        callbacks=[es],
        verbose=0
    )

    best_epoch = int(np.argmin(history.history["val_loss"]))
    best_train_loss = float(history.history["loss"][best_epoch])
    best_val_loss   = float(history.history["val_loss"][best_epoch])

    if label == "":
        print(f"\nFinal train loss: {best_train_loss}")
        print(f"Final val loss  : {best_val_loss}")
    else:
        print(f"\nFinal train loss ({label}): {best_train_loss}")
        print(f"Final val loss  ({label}): {best_val_loss}")

    return model

In [74]:

sgd  = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, clipnorm=1.0)
adam = tf.keras.optimizers.Adam(learning_rate=0.1, clipnorm=1.0)

model_sgd  = train_and_report(sgd,  label="SGD")
model_adam = train_and_report(adam, label="Adam")

last_12_vitals = df[features].tail(lookback).values
last_12_scaled = x_scaler.transform(last_12_vitals).reshape(1, lookback, F)

pred_sgd_scaled  = model_sgd.predict(last_12_scaled,  verbose=0)[0]  # (6,)
pred_adam_scaled = model_adam.predict(last_12_scaled, verbose=0)[0]  # (6,)

# invert scaling to original risk_score scale
pred_sgd  = y_scaler.inverse_transform(pred_sgd_scaled.reshape(-1, 1)).reshape(-1)
pred_adam = y_scaler.inverse_transform(pred_adam_scaled.reshape(-1, 1)).reshape(-1)

last_12_risk = df[target].tail(lookback).values

print("\nLast 12 hours risk_score:", np.round(last_12_risk, 2))
print("Predicted risk next 6 hours (SGD) : ", np.round(pred_sgd, 2))
print("Predicted risk next 6 hours (Adam): ", np.round(pred_adam, 2))


d:\Training\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Final train loss (SGD): 0.0582413524389267
Final val loss  (SGD): 0.008881419897079468

Final train loss (Adam): 0.38756808638572693
Final val loss  (Adam): 0.012296125292778015

Last 12 hours risk_score: [25.91 27.54 28.48 29.38 30.77 31.89 32.59 30.1  29.2  26.2  25.97 26.41]
Predicted risk next 6 hours (SGD) :  [30.44 31.99 27.89 30.76 31.32 26.78]
Predicted risk next 6 hours (Adam):  [31.89 28.34 31.48 28.73 31.08 28.15]
